In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as n


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0


In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
    model=keras.Sequential([
      keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu',
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
          activation='relu'
      ),
      keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
    return model

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 3.4MB/s 


In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch( build_model, objective='val_accuracy', 
                     max_trials=2, directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 12m 22s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.1054999977350235
Total elapsed time: 00h 12m 22s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |96                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |32                
conv_2_kernel     |5                 |3                 
dense_1_units     |80                |48                
learning_rate     |0.001             |0.01              

Epoch 1/3
  95/1688 [>.............................] - ETA: 10:34 - loss: 2.3027 - accuracy: 0.1013

KeyboardInterrupt: ignored

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        27680     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                884784    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 913,914
Trainable params: 913,914
Non-trainable params: 0
_________________________________________________________________
